# SBS 평가: Denoising (학습 모델) + Deconvolution (최소제곱법)

이 노트북은 Step-by-Step 파이프라인의 중간 성능을 확인하기 위해 제작되었습니다.
- **1단계 (Denoising)**: 학습된 DnCNN 모델을 사용하여 노이즈를 제거합니다.
- **2단계 (Deconvolution)**: 고전적인 이미지 복원 기법인 최소제곱법(Least Squares)을 사용하여 Deconvolution을 수행합니다.

이를 통해 전체 SBS 파이프라인(DnCNN + Unet)의 학습이 완료되기 전에 Denoising 모델의 성능과 한계를 잠정적으로 평가할 수 있습니다.


In [ ]:
# @title 1. 환경 설정
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 프로젝트 폴더 경로 설정 (본인의 환경에 맞게 수정)
import os
PROJECT_PATH = "/content/drive/MyDrive/Data Scientist/Project/Week5/week5"
os.chdir(PROJECT_PATH)

# 시스템 경로에 프로젝트 루트 추가
import sys
sys.path.append(PROJECT_PATH)

print(f"Current working directory: {os.getcwd()}")


In [ ]:
# @title 2. 설정값 정의
# @markdown ---
# @markdown ### **1. 실행 모드 선택**
# @markdown - **Robust**: 5개 방향을 모두 테스트하여 이미지별 최적 결과를 자동으로 찾습니다. (권장)
# @markdown - **Manual**: 아래에 지정된 단일 방향으로만 모든 이미지를 복원합니다. (실험용)
EVALUATION_MODE = "Robust (All Directions)" # @param ["Robust (All Directions)", "Manual (Single Direction)"]
# @markdown ---
# @markdown ### **2. 필수 경로 설정**
# @markdown Denoising 모델의 체크포인트 파일 경로를 지정하세요.
DENOISING_CKPT_PATH = "logs_sbs_denoising_dncnn/00001_train/checkpoints/checkpoint_best.ckpt" # @param {type:"string"}
# @markdown ---
# @markdown ### **3. 데이터 및 결과 폴더 설정**
# @markdown 테스트 데이터셋과 결과 파일을 저장할 폴더를 지정하세요.
TEST_DATA_PATH = "dataset/test_y" # @param {type:"string"}
RESULT_DIR = "result_sbs_denoising_ls" # @param {type:"string"}
# @markdown ---
# @markdown ### **4. 최소제곱법 하이퍼파라미터**
# @markdown Regularization 강도 (0에 가까울수록 강한 복원, 클수록 노이즈 억제)
LAMBDA = 1e-3 # @param {type:"number"}
# @markdown ---
# @markdown ### **5. (Manual 모드 전용) B0 방향 수동 설정**
# @markdown `EVALUATION_MODE`가 `Manual`일 때만 사용됩니다.
B0_DIR_X = 0.309 # @param {type:"number"}
B0_DIR_Y = -0.9511 # @param {type:"number"}
MANUAL_B0_DIRECTION = (B0_DIR_X, B0_DIR_Y)
# @markdown ---
print(f"✅ Evaluation Mode: {EVALUATION_MODE}")
print(f"Denoising Checkpoint: {DENOISING_CKPT_PATH}")
print(f"Test Data: {TEST_DATA_PATH}")
print(f"Result Directory: {RESULT_DIR}")
print(f"Least Squares Lambda: {LAMBDA}")
if EVALUATION_MODE == 'Manual (Single Direction)':
    print(f"Manual B0 Direction: {MANUAL_B0_DIRECTION}")


In [ ]:
# @title 3. 최소제곱법(Least Squares) 복원 함수 구현
import torch
import torch.fft as fft
from dataset.forward_simulator import ForwardSimulator
from params import config as global_config, conv_directions

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ForwardSimulator를 한 번만 초기화
H, W = global_config.image_size
forward_simulator = ForwardSimulator(image_dims=(H, W))

# 💡 수정: 선택된 EVALUATION_MODE에 따라 커널을 생성
if EVALUATION_MODE == "Robust (All Directions)":
    dipole_kernels_k = [
        forward_simulator._create_dipole_kernel(B0_direction=direction).to(device)
        for direction in conv_directions
    ]
    print(f"✅ Robust Mode: Successfully created {len(dipole_kernels_k)} dipole kernels.")
else: # Manual (Single Direction)
    dipole_kernels_k = [
        forward_simulator._create_dipole_kernel(B0_direction=MANUAL_B0_DIRECTION).to(device)
    ]
    print(f"✅ Manual Mode: Successfully created 1 dipole kernel for direction {MANUAL_B0_DIRECTION}.")


def least_squares_deconv(denoised_tensor: torch.Tensor, kernel_k: torch.Tensor, lambda_reg: float) -> torch.Tensor:
    """
    Performs Least Squares deconvolution in the Fourier domain.
    """
    # 입력 이미지를 k-space로 변환
    denoised_k = fft.fftn(denoised_tensor, dim=(-2, -1))
    
    # 최소제곱법 공식 적용
    kernel_k_conj = torch.conj(kernel_k)
    kernel_mag_sq = torch.abs(kernel_k)**2
    
    numerator = kernel_k_conj * denoised_k
    denominator = kernel_mag_sq + lambda_reg
    
    # 0으로 나누는 것을 방지
    denominator[denominator == 0] = 1.0
    
    deconv_k = numerator / denominator
    
    # 다시 이미지 공간으로 변환
    deconv_image = fft.ifftn(deconv_k, dim=(-2, -1))
    
    # 실수부만 반환
    return deconv_image.real

print("Least Squares deconvolution function is ready.")



In [ ]:
# @title 4. Denoising 모델 로드
from pathlib import Path
from code_denoising.core_funcs import get_model, ModelType
from params import dncnnconfig

print(f"Loading Denoising checkpoint from: {DENOISING_CKPT_PATH}")

try:
    denoising_ckpt_path = Path(DENOISING_CKPT_PATH)
    if not denoising_ckpt_path.exists():
        raise FileNotFoundError(f"Denoising checkpoint not found at {denoising_ckpt_path}")

    denoising_checkpoint = torch.load(denoising_ckpt_path, map_location=device)
    
    # 모델 타입 및 설정 할당
    global_config.model_type = denoising_checkpoint.get("model_type", "dncnn")
    global_config.model_config = dncnnconfig
    
    # 모델 생성 및 state_dict 로드
    denoising_network = get_model(global_config).to(device)
    denoising_network.load_state_dict(denoising_checkpoint['model_state_dict'])
    denoising_network.eval()
    
    print(f"Successfully loaded Denoising model ({global_config.model_type}).")

except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    denoising_network = None


In [ ]:
# @title 5. 파이프라인 실행 및 결과 저장
import shutil
from tqdm.notebook import tqdm
import numpy as np
from torch.utils.data import DataLoader
from code_denoising.datawrapper.datawrapper import DataKey, get_data_wrapper_loader, LoaderConfig

if denoising_network:
    # --- 기존 결과 폴더 삭제 ---
    result_path = Path(RESULT_DIR)
    if result_path.exists():
        print(f"Removing old '{RESULT_DIR}' directory...")
        shutil.rmtree(result_path)
    result_path.mkdir(parents=True, exist_ok=True)
    
    # --- 데이터 로더 설정 ---
    loader_cfg: LoaderConfig = {
        "data_type": global_config.data_type,
        "batch": 8, # GPU 메모리에 맞춰 배치 크기 조절 가능
        "num_workers": 2,
        "shuffle": False,
        "augmentation_mode": 'none',
        "training_phase": 'end_to_end',
        "noise_type": global_config.noise_type,
        "noise_levels": global_config.noise_levels,
        "conv_directions": global_config.conv_directions
    }
    data_loader, _ = get_data_wrapper_loader(
        file_path=[TEST_DATA_PATH],
        training_mode=False,
        data_wrapper_class='controlled',
        **loader_cfg
    )

    if not data_loader:
        print(f"Failed to create data loader from {TEST_DATA_PATH}. No data found?")
    else:
        print(f"\n[Phase 1/1] Creating result files from Denoising + Robust Least Squares pipeline...")
        with torch.no_grad():
            for data in tqdm(data_loader, desc="Processing images"):
                image_noise = data[DataKey.image_noise].to(device)
                filenames = data[DataKey.name]

                # Step 1: Denoising (학습 모델)
                denoised_image_batch = denoising_network(image_noise)

                # 💡 수정: 배치 내 각 이미지에 대해 개별적으로 최적 방향 탐색
                for i in range(denoised_image_batch.shape[0]):
                    single_denoised_tensor = denoised_image_batch[i:i+1]
                    
                    candidate_results = []
                    candidate_variances = []

                    # Step 2: 5개 커널 모두에 대해 Deconvolution 수행
                    for kernel_k in dipole_kernels_k:
                        deconv_result = least_squares_deconv(single_denoised_tensor, kernel_k, LAMBDA)
                        candidate_results.append(deconv_result)
                        candidate_variances.append(torch.var(deconv_result).item())
                    
                    # Step 3: 분산이 가장 높은 결과를 최적해로 선택
                    best_result_index = np.argmax(candidate_variances)
                    final_image_tensor = candidate_results[best_result_index]

                    # 결과 저장
                    pred_np = final_image_tensor.squeeze().cpu().numpy()
                    base_filename = Path(filenames[i]).stem
                    np.save(result_path / f"{base_filename}.npy", pred_np)
        
        print(f"\nFinished! Results are saved in '{RESULT_DIR}'.")
        print("You can now run the 'evaluate.ipynb' notebook to calculate the scores.")
else:
    print("\nSkipping pipeline execution because the model failed to load.")
